In [1]:
import torch
import torchvision
import torchvision.transforms.v2 as transforms

from going_modular.module import data_loader, engine, utils

In [2]:
# Set seeds
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [3]:
basedir = "./"
train_dir = basedir + "data/pizza_steak_sushi/train"
test_dir = basedir + "data/pizza_steak_sushi/test"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Automatic Transforms
weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT # DEFAULT = best performing weights
auto_transforms = weights.transforms()

In [5]:
train_dataloader, test_dataloader, classnames = data_loader.create_dataloaders(train_dir=train_dir, test_dir=test_dir,transform=auto_transforms, batch_size=32)

train_dataloader, test_dataloader, classnames

(<torch.utils.data.dataloader.DataLoader at 0x7ff637f04b50>,
 ['pizza', 'steak', 'sushi'])

In [6]:
model = torchvision.models.efficientnet_b1(weights=weights)

In [7]:
for param in model.features.parameters():
    param.requires_grad = False

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, out_features=len(classnames),bias=True))

In [8]:
from torchinfo import summary

# Get a summary of the model (uncomment for full output)
summary(model, 
        input_size=(32, 3, 224, 224),
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

/home/heyia/miniforge3/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [9]:
from torch.utils.tensorboard import SummaryWriter
from timeit import default_timer as timer 

In [10]:
# writer = SummaryWriter()
# writer = None

In [18]:
writer = utils.create_writer(experiment_name="data_10", model_name = "effnetb1", extra="10_epochs")

NUM_EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

[INFO] Created SummaryWriter, saving to: runs/2023-10-23/data_10/effnetb1/10_epochs...


In [19]:
start_time = timer()

# Start training with help from engine.py
results = engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device,
             writer=writer)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0708 | train_acc: 0.5000 | test_loss: 0.9853 | test_acc: 0.7216
Epoch: 2 | train_loss: 0.9405 | train_acc: 0.8906 | test_loss: 0.9166 | test_acc: 0.9167
Epoch: 3 | train_loss: 0.8422 | train_acc: 0.9258 | test_loss: 0.8554 | test_acc: 0.8968
Epoch: 4 | train_loss: 0.8202 | train_acc: 0.7891 | test_loss: 0.7930 | test_acc: 0.8561
Epoch: 5 | train_loss: 0.7893 | train_acc: 0.7852 | test_loss: 0.7564 | test_acc: 0.8864
Epoch: 6 | train_loss: 0.7779 | train_acc: 0.8164 | test_loss: 0.7143 | test_acc: 0.8864
Epoch: 7 | train_loss: 0.6536 | train_acc: 0.8203 | test_loss: 0.6628 | test_acc: 0.9479
Epoch: 8 | train_loss: 0.6698 | train_acc: 0.8242 | test_loss: 0.6630 | test_acc: 0.8968
Epoch: 9 | train_loss: 0.6113 | train_acc: 0.8203 | test_loss: 0.6352 | test_acc: 0.8968
Epoch: 10 | train_loss: 0.5887 | train_acc: 0.8359 | test_loss: 0.6252 | test_acc: 0.9072
[INFO] Total training time: 74.378 seconds


In [13]:
model.state_dict().keys()
model.features.state_dict().keys()
model.classifier.state_dict().keys()

odict_keys(['1.weight', '1.bias'])

### View Tensorboard

In [25]:
# %load_ext tensorboard
# %tensorboard --logdir runs

In [20]:
num_epochs = [5,10]

### Systematic Experiment Tracking

In [21]:
# %%time

set_seeds(seed=42)
experiment_number = 0
for epochs in num_epochs: 

    experiment_number += 1
    print(f"[INFO] Experiment number: {experiment_number}")
    print(f"[INFO] Number of epochs: {epochs}")  


    model = torchvision.models.efficientnet_b1(weights=weights)

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=0.2, inplace=True), 
        torch.nn.Linear(in_features=1280, out_features=len(classnames),bias=True))

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

   
    engine.train(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader, 
            optimizer=optimizer,
            loss_fn=loss_fn,
            epochs=epochs,
            device=device,
            writer=utils.create_writer(experiment_name="eff_nets", model_name=model._get_name(), extra=f"{epochs}_epochs"))
    
    save_filepath = f"07_{model._get_name()}_{epochs}_epochs.pth"
    utils.save_model(model=model, target_dir="models", model_name=save_filepath)
    
    print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Number of epochs: 5
[INFO] Created SummaryWriter, saving to: runs/2023-10-23/eff_nets/EfficientNet/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0273 | train_acc: 0.6133 | test_loss: 1.0086 | test_acc: 0.5578
Epoch: 2 | train_loss: 0.9573 | train_acc: 0.7109 | test_loss: 0.9198 | test_acc: 0.7225
Epoch: 3 | train_loss: 0.8363 | train_acc: 0.7539 | test_loss: 0.8523 | test_acc: 0.7850
Epoch: 4 | train_loss: 0.7455 | train_acc: 0.8086 | test_loss: 0.7955 | test_acc: 0.7850
Epoch: 5 | train_loss: 0.8198 | train_acc: 0.7969 | test_loss: 0.7242 | test_acc: 0.7955
[INFO] Saving model to: models/07_EfficientNet_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Number of epochs: 10
[INFO] Created SummaryWriter, saving to: runs/2023-10-23/eff_nets/EfficientNet/10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0718 | train_acc: 0.4883 | test_loss: 1.0048 | test_acc: 0.6515
Epoch: 2 | train_loss: 0.9449 | train_acc: 0.8008 | test_loss: 0.9129 | test_acc: 0.8561
Epoch: 3 | train_loss: 0.8519 | train_acc: 0.9219 | test_loss: 0.8441 | test_acc: 0.9271
Epoch: 4 | train_loss: 0.8169 | train_acc: 0.8047 | test_loss: 0.8111 | test_acc: 0.8864
Epoch: 5 | train_loss: 0.7694 | train_acc: 0.8047 | test_loss: 0.7559 | test_acc: 0.8864
Epoch: 6 | train_loss: 0.6569 | train_acc: 0.9258 | test_loss: 0.6998 | test_acc: 0.8864
Epoch: 7 | train_loss: 0.6485 | train_acc: 0.9062 | test_loss: 0.6795 | test_acc: 0.8864
Epoch: 8 | train_loss: 0.5700 | train_acc: 0.9219 | test_loss: 0.6504 | test_acc: 0.8864
Epoch: 9 | train_loss: 0.5394 | train_acc: 0.9062 | test_loss: 0.6422 | test_acc: 0.8258
Epoch: 10 | train_loss: 0.5035 | train_acc: 0.9180 | test_loss: 0.6353 | test_acc: 0.7945
[INFO] Saving model to: models/07_EfficientNet_10_epochs.pth
------------------------------------------------

In [24]:
# %load_ext tensorboard
# %tensorboard --logdir runs

In [ ]:
# Analyse and load the best model and then predict it ...

# # Check the model file size
# from pathlib import Path

# # Get the model size in bytes then convert to megabytes
# effnetb2_model_size = Path(best_model_path).stat().st_size // (1024*1024)
# print(f"EfficientNetB2 feature extractor model size: {effnetb2_model_size} MB")